In [1]:
import pandas as pd
import numpy as np
from sup_data_to_fasta import load_xlsx, map_raw_data_to_queries_and_expected

In [2]:
esp_df = pd.read_csv('../output/ESP-OD2w11_results.csv').drop(columns=['Unnamed: 0'])

/tmp/ipykernel_741036/1424074734.py:1: DtypeWarning: Columns (92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  esp_df = pd.read_csv('../output/ESP-OD2w11_results.csv').drop(columns=['Unnamed: 0'])


In [3]:
len(esp_df)

298003

In [4]:
total_alignments = len(esp_df[esp_df['qseqid'].notnull()])
not_aligned = len(esp_df[esp_df['qseqid'].isnull()])
print(f"Number of alignments in ESP-OD2w11: {total_alignments}")
print(f"Not aligned in ESP-OD2w11: {not_aligned}")

Number of alignments in ESP-OD2w11: 298003
Not aligned in ESP-OD2w11: 0


In [17]:
len(esp_df[esp_df['sseq_cds'] == esp_df['sseq_genome']])

14213

In [18]:
cds_columns = esp_df.filter(like='_cds')
cds_columns


,qgi_cds,qacc_cds,qaccver_cds,qlen_cds,sseqid_cds,sallseqid_cds,sgi_cds,sallgi_cds,sacc_cds,saccver_cds,...,staxids_cds,sscinames_cds,scomnames_cds,sblastnames_cds,sskingdoms_cds,stitle_cds,salltitles_cds,sstrand_cds,qcovs_cds,qcovhsp_cds
0,0.0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
1,0.0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212.0,lcl|NC_016810.1_cds_WP_001520210.1_484,lcl|NC_016810.1_cds_WP_001520210.1_484,0.0,0.0,lcl|NC_016810.1_cds_WP_001520210.1_484,lcl|NC_016810.1_cds_WP_001520210.1_484,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,plus,13.0,13.0
2,0.0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212.0,lcl|NC_017718.1_cds_WP_000142425.1_4636,lcl|NC_017718.1_cds_WP_000142425.1_4636,0.0,0.0,lcl|NC_017718.1_cds_WP_000142425.1_4636,lcl|NC_017718.1_cds_WP_000142425.1_4636,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,plus,7.0,7.0
3,0.0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212.0,lcl|NC_016810.1_cds_WP_000809815.1_3388,lcl|NC_016810.1_cds_WP_000809815.1_3388,0.0,0.0,lcl|NC_016810.1_cds_WP_000809815.1_3388,lcl|NC_016810.1_cds_WP_000809815.1_3388,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,plus,8.0,8.0
4,0.0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297998,0.0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59.0,lcl|NC_016810.1_cds_WP_001181280.1_854,lcl|NC_016810.1_cds_WP_001181280.1_854,0.0,0.0,lcl|NC_016810.1_cds_WP_001181280.1_854,lcl|NC_016810.1_cds_WP_001181280.1_854,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,plus,22.0,22.0
297999,0.0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59.0,lcl|NC_016810.1_cds_WP_000057014.1_677,lcl|NC_016810.1_cds_WP_000057014.1_677,0.0,0.0,lcl|NC_016810.1_cds_WP_000057014.1_677,lcl|NC_016810.1_cds_WP_000057014.1_677,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,minus,32.0,32.0
298000,0.0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59.0,lcl|NC_016810.1_cds_WP_001526312.1_392,lcl|NC_016810.1_cds_WP_001526312.1_392,0.0,0.0,lcl|NC_016810.1_cds_WP_001526312.1_392,lcl|NC_016810.1_cds_WP_001526312.1_392,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,plus,27.0,27.0
298001,0.0,b25c534f507b4a16866e9afeff4ae768,b25c534f507b4a16866e9afeff4ae768,51.0,lcl|NC_016810.1_cds_WP_010988985.1_824,lcl|NC_016810.1_cds_WP_010988985.1_824,0.0,0.0,lcl|NC_016810.1_cds_WP_010988985.1_824,lcl|NC_016810.1_cds_WP_010988985.1_824,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,plus,33.0,33.0


In [19]:
esp_df['stitle_cds'].isnull().sum()

20

In [21]:
esp_df['gene'] = esp_df['stitle_cds'].str.extract(r'gene=(\w+)')
esp_df['locus_tag'] = esp_df['stitle_cds'].str.extract(r'locus_tag=(\w+)')
esp_df['expected_gene'] = esp_df['name'].str.extract(r'(\w+)\(')
esp_df['expected_locus_tag'] = esp_df['name'].str.extract(r'(SL\d+_\d+)')


In [23]:
def was_there_query_not_aligned_w_expected_gene(df):
    grouped = df.query("gene == expected_gene").groupby('query_id')['query_id'].count()
    
    return (grouped == 0).any()

In [24]:
was_there_query_not_aligned_w_expected_gene(esp_df)

False

In [13]:
aligned_w_differ_gene = esp_df.query("expected_gene != gene")

In [14]:
locus_tag_second_number = esp_df.expected_locus_tag.str.extract(r'SL\d+_(\d+)')[0].unique()

In [15]:
esp_df

,qseqid,qgi_genome,qacc_genome,qaccver_genome,qlen_genome,sseqid_genome,sallseqid_genome,sgi_genome,sallgi_genome,sacc_genome,...,Fisher's exact test p-value,type,seq,origin,chimera_idx,query_id,gene,locus_tag,expected_gene,expected_locus_tag
0,8822d12f295f4cbca33753ee0770432b,0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,8822d12f295f4cbca33753ee0770432b,thrL,SL1344_RS00005,thrL,SL1344_0001
1,8822d12f295f4cbca33753ee0770432b,0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,8822d12f295f4cbca33753ee0770432b,htpG,SL1344_RS02480,thrL,SL1344_0001
2,8822d12f295f4cbca33753ee0770432b,0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,8822d12f295f4cbca33753ee0770432b,NaN,SL1344_RS24260,thrL,SL1344_0001
3,8822d12f295f4cbca33753ee0770432b,0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,8822d12f295f4cbca33753ee0770432b,arcB,SL1344_RS17190,thrL,SL1344_0001
4,8822d12f295f4cbca33753ee0770432b,0,8822d12f295f4cbca33753ee0770432b,8822d12f295f4cbca33753ee0770432b,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,8822d12f295f4cbca33753ee0770432b,thrL,SL1344_RS00005,thrL,SL1344_0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297998,21b140528aa943d8b49bd8c7587561b0,0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,21b140528aa943d8b49bd8c7587561b0,NaN,SL1344_RS04385,ArcZ,NaN
297999,21b140528aa943d8b49bd8c7587561b0,0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,21b140528aa943d8b49bd8c7587561b0,tcuC,SL1344_RS03480,ArcZ,NaN
298000,21b140528aa943d8b49bd8c7587561b0,0,21b140528aa943d8b49bd8c7587561b0,21b140528aa943d8b49bd8c7587561b0,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,21b140528aa943d8b49bd8c7587561b0,sbcC,SL1344_RS02020,ArcZ,NaN
298001,b25c534f507b4a16866e9afeff4ae768,0,b25c534f507b4a16866e9afeff4ae768,b25c534f507b4a16866e9afeff4ae768,51,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,9.910000e-45,sRNA,AAAAAGTAACATGGGGCTGTATTTCCCCCTCTGGATGAGGGGGCTT...,RNA2,854,b25c534f507b4a16866e9afeff4ae768,NaN,SL1344_RS04245,STnc740,NaN
